In [1]:
!git clone --branch cohda https://github.com/Ellariel/mango_hybit/

Cloning into 'mango_hybit'...
remote: Enumerating objects: 1380, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1380 (delta 34), reused 62 (delta 25), pack-reused 1301 (from 1)
Receiving objects: 100% (1380/1380), 31.89 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (846/846), done.


In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
def square(data, col='steptime', func=np.mean, alg=['cohda', 'default']):
    a = np.arange(len(alg) ** 2, dtype=float).reshape(len(alg), len(alg))
    for x, i in enumerate(alg):
      for y, j in enumerate(alg):
        df = data[(i, j)]
        a[x, y] = func(data[(i, j)][col])
    return pd.DataFrame(a, index=alg, columns=alg)

In [4]:
base_dir = '/content/mango_hybit/examples/cohda/results'
alg = ['cohda', 'default']
data = {}
for i in alg: # --within {i} --between {j}
  for j in alg:
    df = pd.read_csv(os.path.join(base_dir, f"{i}_{j}_results.csv")).rename(columns={
        'MASSCA-0.MosaikAgent-steptime' : 'steptime',
        'GridSim-0.ExternalGrid-0-value' : 'ExternalGrid',
    })
    df['steptime'] = df['steptime'].shift(1)
    df['production'] = df['GridSim-0.Bus-3-value'] + df['GridSim-0.Bus-5-value']
    df['consumption'] = df['GridSim-0.Bus-2-value'] + df['GridSim-0.Bus-4-value']
    df = df.iloc[1::2]
    data[(i, j)] = df

In [5]:
table = []
for i in alg:
  for j in alg:
    df = data[(i, j)]
    table.append({'within' : i,
                  'between' : j,
                  'runtime' : f"{df['steptime'].mean():.2f}",
                  'grid balance' : f"{df['ExternalGrid'].sum():.2f}",
                  'total production' : f"{df['production'].sum():.2f}",
                  'total consumption' : f"{df['consumption'].sum():.2f}"})
pd.DataFrame(table)

,within,between,runtime,grid balance,total production,total consumption
0,cohda,cohda,3.90,2.89,-38.00,40.68
1,cohda,default,8.83,7.84,-33.03,40.68
2,default,cohda,7.09,-9.17,-50.09,40.68
3,default,default,0.07,0.20,-40.68,40.68


In [7]:
square(data)

,cohda,default
cohda,3.898617,8.831071
default,7.089231,0.074954


In [8]:
square(data, 'ExternalGrid', np.sum)

,cohda,default
cohda,2.886267,7.844355
default,-9.167065,0.201138


In [9]:
square(data, 'production', np.sum)

,cohda,default
cohda,-38.002663,-33.031848
default,-50.092620,-40.682692


In [10]:
square(data, 'consumption', np.sum)

,cohda,default
cohda,40.682692,40.682692
default,40.682692,40.682692
